### Programming Assignment 1 Q2
### Bayesian (Naïve Bayes) Classifier

Group Number : 46

Student 1 : Aryan Singh 19CS30007

Student 2 : Seemant Guruprasad Achari 19CS30057


### Tools

In [1]:
import numpy as np
import pandas as pd

# for pretty printing summary
import json 
     

import matplotlib.pyplot as plt

In [2]:
# A constant to reference the result column
LABEL = 'Response'
# max unique categories to consider
MAX_UNIQUE_CATEGORIES = 20

# tag for categorical attributes
CATEGORICAL_FEATURE = "categorical"

# tag for continuous attributes
CONTINUOUS_FEATURE = "continuous"

In [3]:
# splitting dataset index maintaing the ratio of true and false examples
def train_test_split(df,train_sample=0.5,target_col= LABEL):
    all_indexes=[]
    df_grouped= df.groupby(target_col)
    for x,x_df in df_grouped:
        t = x_df.sample(frac=train_sample,).index
        all_indexes.append(t)
    g = all_indexes[0].values
    for k in all_indexes[1:]:
        g=np.hstack([g,k.values])
        #np.hstack([all_indexes[0].values,all_indexes[1].values])

    train_df = df[df.index.isin(g) ]
    test_df = df[~df.index.isin(g)]
    return train_df, test_df 

In [5]:
# Data read using pandas
df = pd.read_csv('./Dataset_C.csv')
df = df.drop(["id"], axis = 1) # since id doesn't give any information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131689 entries, 0 to 131688
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Gender                131689 non-null  object
 1   Age                   131689 non-null  int64 
 2   Driving_License       131689 non-null  int64 
 3   Region_Code           131689 non-null  int64 
 4   Previously_Insured    131689 non-null  int64 
 5   Vehicle_Age           131689 non-null  object
 6   Vehicle_Damage        131689 non-null  object
 7   Annual_Premium        131689 non-null  int64 
 8   Policy_Sales_Channel  131689 non-null  int64 
 9   Vintage               131689 non-null  int64 
 10  Response              131689 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 11.1+ MB


In [6]:
# Classifying the attributes as continuous or categorical feature
categoricalStringCols = []
print("Analzing dataset")
for k in df.columns:
    if k == 'id':
        continue
    if k == LABEL :
        print("Label     => ", end = "")
    else:
        print("Attribute => ", end = "")
        if(len(df[k].unique()) <= MAX_UNIQUE_CATEGORIES and type(df[k][0]) == str):
            categoricalStringCols.append(k)
    print(f"{k} has {len(df[k].unique())} unique values.", end="")
    if(len(df[k].unique()) <= MAX_UNIQUE_CATEGORIES):
        print("[ ", end = "")
        for val in df[k].unique():
            print(val, end = ", ")
        print("]", end="")
    print()
print(f"Categorical string features are {categoricalStringCols}")
print()

Analzing dataset
Attribute => Gender has 2 unique values.[ Male, Female, ]
Attribute => Age has 66 unique values.
Attribute => Driving_License has 2 unique values.[ 1, 0, ]
Attribute => Region_Code has 53 unique values.
Attribute => Previously_Insured has 2 unique values.[ 0, 1, ]
Attribute => Vehicle_Age has 3 unique values.[ > 2 Years, 1-2 Year, < 1 Year, ]
Attribute => Vehicle_Damage has 2 unique values.[ Yes, No, ]
Attribute => Annual_Premium has 36617 unique values.
Attribute => Policy_Sales_Channel has 140 unique values.
Attribute => Vintage has 290 unique values.
Label     => Response has 2 unique values.[ 1, 0, ]
Categorical string features are ['Gender', 'Vehicle_Age', 'Vehicle_Damage']



In [7]:
print("---------------------Task-1-Started----------------------------")
print("Encoding of categorical feature started")
vehicle_age_dict = {
    '< 1 Year' : 0,
    '1-2 Year' : 1,
    '> 2 Years': 2
}
df["Vehicle_Age_ordinal"] = df.Vehicle_Age.map(vehicle_age_dict)

gender_dict = {
    'Male': 0,
    'Female': 1
}
df["Gender_encoded"] = df.Gender.map(gender_dict)

vehicle_damage_dict = {
    'No' : 0,
    'Yes' : 1
}

df["Vehicle_Damage_encoded"] = df.Vehicle_Damage.map(vehicle_damage_dict)
print("Encoding of categorical feature finished")


---------------------Task-1-Started----------------------------
Encoding of categorical feature started
Encoding of categorical feature finished


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131689 entries, 0 to 131688
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Gender                  131689 non-null  object
 1   Age                     131689 non-null  int64 
 2   Driving_License         131689 non-null  int64 
 3   Region_Code             131689 non-null  int64 
 4   Previously_Insured      131689 non-null  int64 
 5   Vehicle_Age             131689 non-null  object
 6   Vehicle_Damage          131689 non-null  object
 7   Annual_Premium          131689 non-null  int64 
 8   Policy_Sales_Channel    131689 non-null  int64 
 9   Vintage                 131689 non-null  int64 
 10  Response                131689 non-null  int64 
 11  Vehicle_Age_ordinal     131689 non-null  int64 
 12  Gender_encoded          131689 non-null  int64 
 13  Vehicle_Damage_encoded  131689 non-null  int64 
dtypes: int64(11), object(3)
memory usage

In [9]:
# Dropping old non-encoded columns

df = df.drop(['Gender', 'Vehicle_Age', 'Vehicle_Damage'], axis = 1)
print(f"Replaced the {categoricalStringCols} columns with their encoded versions")

Replaced the ['Gender', 'Vehicle_Age', 'Vehicle_Damage'] columns with their encoded versions


In [10]:
print(df.head())

   Age  Driving_License  Region_Code  Previously_Insured  Annual_Premium  \
0   44                1           28                   0           40454   
1   76                1            3                   0           33536   
2   47                1           28                   0           38294   
3   21                1           11                   1           28619   
4   29                1           41                   1           27496   

   Policy_Sales_Channel  Vintage  Response  Vehicle_Age_ordinal  \
0                    26      217         1                    2   
1                    26      183         0                    1   
2                    26       27         1                    2   
3                   152      203         0                    0   
4                   152       39         0                    0   

   Gender_encoded  Vehicle_Damage_encoded  
0               0                       1  
1               0                       0  
2       

In [11]:
def determineTypeOfFeature(df):
    print("Determining type of feature")
    feature_types = {}
    n_unique_values_treshold = MAX_UNIQUE_CATEGORIES
    for feature in df.columns:
        if feature == 'id':
            continue
        if feature != LABEL:
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types[feature] = CATEGORICAL_FEATURE
            else:
                feature_types[feature] = CONTINUOUS_FEATURE
    
    return feature_types

# Generating a constant map which we will use ahead
FEATURE_TYPES = determineTypeOfFeature(df)
for k, v in FEATURE_TYPES.items():
    print(f"Attr {k} is {v}")

Determining type of feature
Attr Age is continuous
Attr Driving_License is categorical
Attr Region_Code is continuous
Attr Previously_Insured is categorical
Attr Annual_Premium is continuous
Attr Policy_Sales_Channel is continuous
Attr Vintage is continuous
Attr Vehicle_Age_ordinal is categorical
Attr Gender_encoded is categorical
Attr Vehicle_Damage_encoded is categorical


In [12]:
# Generating Random split with fixed ratio, for training and testing
train_set, test_set = train_test_split(df, .8)
print("---------------------Task-1-Completed--------------------------")

---------------------Task-1-Completed--------------------------


#### Removing Outliers

In [13]:
print("---------------------Task-2-Started----------------------------")
print(f"Inititial Dataset count {df.shape[0]}")
for feature in FEATURE_TYPES.keys():
    print()
    print(f"Feature {feature}")
    feature_mean = df[feature].mean()
    feature_std = df[feature].std()
    print(f"Mean               :{feature_mean}")
    print(f"Standard Deviation :{feature_std}")
    
    lower_bound = feature_mean - 3* feature_std
    upper_bound = feature_mean + 3*feature_std
    print(f"Lower cutoff = {lower_bound}")
    print(f"Upper cutoff = {upper_bound}")

    
    print(f"Outlier count = {df[(df[feature]<lower_bound) | (df[feature]>upper_bound)].shape[0]}")
    print("Filtering the data set")
    df = df[(df[feature]>=lower_bound) & (df[feature]<=upper_bound)]
print(f"Final dataset count after removing outliers = {df.shape[0]}")
print()

---------------------Task-2-Started----------------------------
Inititial Dataset count 131689

Feature Age
Mean               :38.8106068084654
Standard Deviation :15.494455932192853
Lower cutoff = -7.672760988113161
Upper cutoff = 85.29397460504396
Outlier count = 0
Filtering the data set

Feature Driving_License
Mean               :0.9979193402638034
Standard Deviation :0.04556694370158682
Lower cutoff = 0.8612185091590429
Upper cutoff = 1.1346201713685637
Outlier count = 274
Filtering the data set

Feature Region_Code
Mean               :26.407731233116465
Standard Deviation :13.222132003580189
Lower cutoff = -13.258664777624105
Upper cutoff = 66.07412724385703
Outlier count = 0
Filtering the data set

Feature Previously_Insured
Mean               :0.45802229578054254
Standard Deviation :0.4982366519278903
Lower cutoff = -1.0366876600031283
Upper cutoff = 1.9527322515642136
Outlier count = 0
Filtering the data set

Feature Annual_Premium
Mean               :30472.05259673553
Standa

In [14]:
toDropFeature = []
for feature in FEATURE_TYPES.keys():
    unique_count = df[feature].unique().shape[0]
    if unique_count <= 1:
        print(f"Removing feature {feature}, since the unique value left in the samples = {unique_count}")
        toDropFeature.append(feature)
        
df = df.drop(toDropFeature, axis = 1)


Removing feature Driving_License, since the unique value left in the samples = 1


In [15]:
# Generating a final constant map which we will use in futher analysis
FINAL_FEATURE_TYPES = determineTypeOfFeature(df)
print("Final features")
for k, v in FINAL_FEATURE_TYPES.items():
    print(f"Feature {k} is {v}")

Determining type of feature
Final features
Feature Age is continuous
Feature Region_Code is continuous
Feature Previously_Insured is categorical
Feature Annual_Premium is continuous
Feature Policy_Sales_Channel is continuous
Feature Vintage is continuous
Feature Vehicle_Age_ordinal is categorical
Feature Gender_encoded is categorical
Feature Vehicle_Damage_encoded is categorical


In [16]:
print("Normalizing the data set, using min-max normalization")
normalized_df=(df-df.min())/(df.max()-df.min())
print("Normalization complete")

Normalizing the data set, using min-max normalization
Normalization complete


In [17]:
print(normalized_df.head())
print("---------------------Task-2-Completed--------------------------")

        Age  Region_Code  Previously_Insured  Annual_Premium  \
0  0.369231     0.538462                 0.0        0.477570   
1  0.861538     0.057692                 0.0        0.390222   
2  0.415385     0.538462                 0.0        0.450297   
3  0.015385     0.211538                 1.0        0.328140   
4  0.138462     0.788462                 1.0        0.313961   

   Policy_Sales_Channel   Vintage  Response  Vehicle_Age_ordinal  \
0              0.154321  0.716263       1.0                  1.0   
1              0.154321  0.598616       0.0                  0.5   
2              0.154321  0.058824       1.0                  1.0   
3              0.932099  0.667820       0.0                  0.0   
4              0.932099  0.100346       0.0                  0.0   

   Gender_encoded  Vehicle_Damage_encoded  
0             0.0                     1.0  
1             0.0                     0.0  
2             0.0                     1.0  
3             0.0             

# Task 3

In [18]:
# Calculating Prior for all classes of label
def calculate_prior(train_data, label):
    classes = train_data[label].unique()
    prior = {}
    for c in classes:
        prior[c] = (len(train_data[train_data[label]==c])/len(train_data))
    return prior

# Calculating P(X = x | Label = Y) for categorical features
def calculate_likelihood_categorical(summary, feature, feature_value, label, Y, laplaceCorrection = False, uniqueFeatureCount = 0):
#     Data for label = Y
#     df_given_y = train_data[train_data[label] == Y]
#     p_x_given_y = len(df_given_y[df_given_y[feature] == feature_value]) / len(df_given_y)
    numerator = summary[feature][Y][feature_value][0]
    denominator = summary[feature][Y][feature_value][1]
    if(laplaceCorrection):
        numerator+=1
        denominator+=uniqueFeatureCount
    p_x_given_y = numerator / denominator
    return p_x_given_y

# Calculating P(X = x  | label = Y) for continous features
def calculate_likelihood_gaussian(summary, feature, feature_value, label, Y):
#     df_given_y = train_data[train_data[label] == Y]
#     mean, std = df_given_y[feature].mean(), df_given_y[feature].std()
    mean, std = summary[feature][Y]['mean'], summary[feature][Y]['std']

    p_x_given_y = (1 / (np.sqrt(2 * np.pi) * std)) *  np.exp(-((feature_value-mean)**2 / (2 * std**2 )))
    return p_x_given_y

In [19]:
# class to feature to summary 
'''
summary[feature][class][mean / std] for continous
summary[feature][class][feature_value] = (tuple) (count of feature=feature value | class , count of feature class)
The second one is count so that we may use the same summary for Laplace correction
'''

def generateSummary(train_data, label):
    result = {}
    classes = train_data[label].unique()
    for feature in FINAL_FEATURE_TYPES.keys():
        if (FINAL_FEATURE_TYPES[feature] == CONTINUOUS_FEATURE):
            # store its std and mean for every label class
            feature_result = {}
            for c in classes:
                df_given_y = train_data[train_data[label] == c]
                mean, std = df_given_y[feature].mean(), df_given_y[feature].std()
                feature_result[c] = {
                    'mean': mean,
                    'std': std
                } 
            result[feature] = feature_result
        else:
#                     Condition when feature is category
            feature_result = {}
            for c in classes:
                class_result = {}
                for feature_value in train_data[feature].unique():
                    df_given_y = train_data[train_data[label] == c]
                    p_x_given_y_tuple = (len(df_given_y[df_given_y[feature] == feature_value]) , len(df_given_y))
                    class_result[feature_value] = p_x_given_y_tuple
                feature_result[c] = class_result
            result[feature] = feature_result
    return result
                    
            

In [20]:
# count the number of unique categorical feature value
count = 0
for feature in FINAL_FEATURE_TYPES.keys():
    if FINAL_FEATURE_TYPES[feature] == CATEGORICAL_FEATURE:
        count += len(normalized_df[feature].unique())
        
UNIQUE_FEATURE_VALUES = count # we shall use this value in laplace correction


In [21]:
print("---------------------Task-3-Started----------------------------")
print(f"unique feature value count = {UNIQUE_FEATURE_VALUES}")
summary = json.dumps(generateSummary(normalized_df, LABEL), indent=2)
print(f"Summary of normalized data : {summary}")


---------------------Task-3-Started----------------------------
unique feature value count = 9
Summary of normalized data : {
  "Age": {
    "1.0": {
      "mean": 0.3592262033499147,
      "std": 0.18642265947548364
    },
    "0.0": {
      "mean": 0.27835371741229487,
      "std": 0.24224011012706498
    }
  },
  "Region_Code": {
    "1.0": {
      "mean": 0.5147213611131136,
      "std": 0.22525781100045275
    },
    "0.0": {
      "mean": 0.5070472588778028,
      "std": 0.25837371647767776
    }
  },
  "Previously_Insured": {
    "1.0": {
      "0.0": [
        15957,
        16005
      ],
      "1.0": [
        48,
        16005
      ]
    },
    "0.0": {
      "0.0": [
        54875,
        114720
      ],
      "1.0": [
        59845,
        114720
      ]
    }
  },
  "Annual_Premium": {
    "1.0": {
      "mean": 0.3579357115665688,
      "std": 0.2083782776534106
    },
    "0.0": {
      "mean": 0.3445268118647404,
      "std": 0.1966454689052933
    }
  },
  "Policy_

In [22]:
def naive_bayes_algo(train_data, test_X, label, laplaceCorrection = False):
    # calculate prior
    prior = calculate_prior(train_data, label)
    summary = generateSummary(train_data, label)
    Y_pred = []
    # loop over every data sample
    for index, x in test_X.iterrows():
        # calculate likelihood
        classes = train_data[label].unique()
        
        likelihood = {}
        for c in classes:
            likelihood[c] = 1
            
        
        for c in classes:
            for feature in FINAL_FEATURE_TYPES.keys():
                if (FINAL_FEATURE_TYPES[feature] == CATEGORICAL_FEATURE):
                    likelihood[c] *= calculate_likelihood_categorical(summary, feature, x[feature], label, c, laplaceCorrection, UNIQUE_FEATURE_VALUES)
                else:
#                     Condition when feature is continous
                    likelihood[c] *= calculate_likelihood_gaussian(summary, feature, x[feature], label, c) 

        # calculate posterior probability (numerator only)
        post_prob = {}
        for c in classes:
            post_prob[c] = 1
        
        for c in classes:
            post_prob[c] = likelihood[c] * prior[c]
#         argmax 
        Y_pred.append(max(post_prob, key=post_prob.get))

    return np.array(Y_pred) 

In [23]:
# Evaluates Accuracy

def GetAccuracy(Y_test, Y_pred, printDetails = True):
    
    accuracy = sum((Y_test == Y_pred)) / len(Y_test)
    if(printDetails):
        print(f"Accuracy  = {accuracy}")
    
    return accuracy

# Evaluates precision for predicting 1
def GetPrecision(Y_test, Y_pred, printDetails = True):
    marked_true_samples = sum(Y_pred == 1)
    correctly_marked = sum((Y_test == Y_pred) & (Y_test == 1))
    precision = 0
    if(marked_true_samples > 0):
        precision = correctly_marked / marked_true_samples
    if(printDetails):
        print(f"Precision = {precision}")
    
    return precision

# Evaluates recall for predicting 1
def GetRecall(Y_test, Y_pred, printDetails = True):
    true_samples = sum(Y_test == 1)
    correctly_marked = sum((Y_test == Y_pred) & (Y_test == 1))
    recall = 0
    if(true_samples >0):
        recall = correctly_marked / true_samples
    if(printDetails):
        print(f"Recall    = {recall}")
    
    return recall

def evaluateAlgo(Y_test, Y_pred, printDetails = True):
    acc = GetAccuracy(Y_test, Y_pred, printDetails)
    pre = GetPrecision(Y_test, Y_pred, printDetails)
    rec = GetRecall(Y_test, Y_pred, printDetails)
    return [acc, pre, rec]


In [24]:
def KFolds(dataframe, k = 10):
    # shuffle the data
    shuffled_df = dataframe.sample(frac=1)    
    count = shuffled_df.shape[0]
    size_of_fold = (count + k-1)//k
    start = 0
    datas = []
    for i in range(k):
        start_idx = i*size_of_fold
        end_idx = (i+1)*size_of_fold
        datas.append(shuffled_df.iloc[start_idx: end_idx, :])
    
    return datas
    
    

In [25]:

def kFoldValidation(dataframe, algorithm, label, k=10, laplaceCorrection = False, _folds = None):
    print(f"{k} fold validation started")
    accuracy = []
    precision = []
    recall = []
    folds = _folds
    if folds == None:
        print("Ola")
        folds = KFolds(dataframe, 10)
    for iteration in range(k):
        print(f"Iter : {iteration+1}  started")
        test = folds[iteration]
        train = pd.concat(folds[0:iteration] + folds[iteration+1:k])

        X_test = test.drop([label], axis = 1)
        Y_test = test[label]
        Y_pred = algorithm(train, test_X=X_test, label = label, laplaceCorrection = laplaceCorrection)
        acc, pre, rec = evaluateAlgo(Y_test, Y_pred)
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
    print()
    print(f"Mean accuracy = {np.mean(accuracy)}")
    print(f"Mean precision = {np.mean(precision)}")  
    print(f"Mean recall = {np.mean(recall)}")
    
    print(f"{k} fold validation finished")
    return folds # for task 4

In [28]:
folds = kFoldValidation(normalized_df, naive_bayes_algo, LABEL, 10)
print("---------------------Task-3-Completed--------------------------")
print()

10 fold validation started
Ola
Iter : 1  started
Accuracy  = 0.7697544557484892
Precision = 0.2887365911799762
Recall    = 0.6086683417085427
Iter : 2  started
Accuracy  = 0.7643999082077565
Precision = 0.29396248534583824
Recall    = 0.5991636798088411
Iter : 3  started
Accuracy  = 0.7677656238047885
Precision = 0.27794292508917956
Recall    = 0.606355382619974
Iter : 4  started
Accuracy  = 0.7611871796833167
Precision = 0.2688622754491018
Recall    = 0.5690747782002535
Iter : 5  started
Accuracy  = 0.7696779622121931
Precision = 0.298105387803434
Recall    = 0.6114146933819065
Iter : 6  started
Accuracy  = 0.7660827660062725
Precision = 0.2798695523273051
Recall    = 0.600127145581691
Iter : 7  started
Accuracy  = 0.7605752313929472
Precision = 0.2921478060046189
Recall    = 0.5988165680473373
Iter : 8  started
Accuracy  = 0.7633289986996099
Precision = 0.2849740932642487
Recall    = 0.61875
Iter : 9  started
Accuracy  = 0.772967184272929
Precision = 0.290147634829768
Recall    = 0.6

In [29]:
print("---------------------Task-4-Started----------------------------")
kFoldValidation(normalized_df, naive_bayes_algo, LABEL, 10, True,  _folds = folds)
print("---------------------Task-4-Finished---------------------------")

---------------------Task-4-Started----------------------------
10 fold validation started
Iter : 1  started
Accuracy  = 0.7697544557484892
Precision = 0.28861061419200956
Recall    = 0.6080402010050251
Iter : 2  started
Accuracy  = 0.7643999082077565
Precision = 0.2937206572769953
Recall    = 0.5979689366786141
Iter : 3  started
Accuracy  = 0.7679186108773809
Precision = 0.2781082688875669
Recall    = 0.606355382619974
Iter : 4  started
Accuracy  = 0.7611871796833167
Precision = 0.2685851318944844
Recall    = 0.5678073510773131
Iter : 5  started
Accuracy  = 0.7696779622121931
Precision = 0.298105387803434
Recall    = 0.6114146933819065
Iter : 6  started
Accuracy  = 0.7663122466151611
Precision = 0.27998812351543945
Recall    = 0.5994914176732359
Iter : 7  started
Accuracy  = 0.7606517249292435
Precision = 0.29163046626122213
Recall    = 0.5958579881656805
Iter : 8  started
Accuracy  = 0.7635584793084984
Precision = 0.28509656961660423
Recall    = 0.618125
Iter : 9  started
Accuracy  =